In this tutorial, you will learn three approaches to dealing with missing values. Then you'll compare the effectiveness of these approaches on a real-world dataset.

### Introduction
There are many ways data can end up with missing values. For example,

    A 2 bedroom house won't include a value for the size of a third bedroom.
    
    A survey respondent may choose not to share his income.
    
    Most machine learning libraries (including scikit-learn) give an error if you try to build a model using data with missing values. So you'll need to choose one of the strategies below.

### Three Approaches
#### 1) A Simple Option: Drop Columns with Missing Values
The simplest option is to drop columns with missing values.

Unless most values in the dropped columns are missing, the model loses access to a lot of (potentially useful!) information with this approach. As an extreme example, consider a dataset with 10,000 rows, where one important column is missing a single entry. This approach would drop the column entirely!

#### 2) A Better Option: Imputation
Imputation fills in the missing values with some number. For instance, we can fill in the mean value along each column.

The imputed value won't be exactly right in most cases, but it usually leads to more accurate models than you would get from dropping the column entirely.

#### 3) An Extension To Imputation¶
Imputation is the standard approach, and it usually works well. However, imputed values may be systematically above or below their actual values (which weren't collected in the dataset). Or rows with missing values may be unique in some other way. In that case, your model would make better predictions by considering which values were originally missing.
                                                                                                                                                  
In this approach, we impute the missing values, as before. And, additionally, for each column with missing entries in the original dataset, we add a new column that shows the location of the imputed entries.

In some cases, this will meaningfully improve results. In other cases, it doesn't help at all.                                                                                                                                      

### Example
In the example, we will work with the Melbourne Housing dataset. Our model will use information such as the number of rooms and land size to predict home price.

We won't focus on the data loading step. Instead, you can imagine you are at a point where you already have the training and validation data in X_train, X_valid, y_train, and y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data
data = pd.read_csv("Dataset/melb_data.csv", index_col=0)
# Select target
y = data.Price
# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

In [2]:
print(data.head())
print('='*100)
print(melb_predictors.head())
print('='*100)
print(X.head())

                     Address  Rooms Type      Price Method SellerG       Date  \
Suburb                                                                          
Abbotsford      85 Turner St      2    h  1480000.0      S  Biggin  3/12/2016   
Abbotsford   25 Bloomburg St      2    h  1035000.0      S  Biggin  4/02/2016   
Abbotsford      5 Charles St      3    h  1465000.0     SP  Biggin  4/03/2017   
Abbotsford  40 Federation La      3    h   850000.0     PI  Biggin  4/03/2017   
Abbotsford       55a Park St      4    h  1600000.0     VB  Nelson  4/06/2016   

            Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  \
Suburb                                                              
Abbotsford       2.5    3067.0       2.0       1.0  1.0     202.0   
Abbotsford       2.5    3067.0       2.0       1.0  0.0     156.0   
Abbotsford       2.5    3067.0       3.0       2.0  0.0     134.0   
Abbotsford       2.5    3067.0       3.0       2.0  1.0      94.0   
Abbotsford       2

In [3]:
print('\t\t\t\tMissing Values \n')
print('melb_predictors \n\n',melb_predictors.isnull().sum())
print('='*110)
print('X \n\n', X.isnull().sum())


				Missing Values 

melb_predictors 

 Address             0
Rooms               0
Type                0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64
X 

 Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64


#### Define Function to Measure Quality of Each Approach
We define a function score_dataset() to compare different approaches to dealing with missing values. This function reports the mean absolute error (MAE) from a random forest model.

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)


#### Score from Approach 1 (Drop Columns with Missing Values)
Since we are working with both training and validation sets, we are careful to drop the same columns in both DataFrames.

In [5]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                    if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

print('='*110)
print('reduced_X_train \n',reduced_X_train.head())
print('='*110)
print('reduced_X_valid \n',reduced_X_valid.head())
print('='*110)
print('cols_with_missing \n',cols_with_missing)

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635
reduced_X_train 
                 Rooms  Distance  Postcode  Bedroom2  Bathroom  Landsize  \
Suburb                                                                    
St Kilda            1       5.0    3182.0       1.0       1.0       0.0   
Williamstown        2       8.0    3016.0       2.0       2.0     193.0   
Sunshine            3      12.6    3020.0       3.0       1.0     555.0   
Glenroy             3      13.0    3046.0       3.0       1.0     265.0   
Sunshine North      3      13.3    3020.0       3.0       1.0     673.0   

                Lattitude  Longtitude  Propertycount  
Suburb                                                
St Kilda        -37.85984    144.9867        13240.0  
Williamstown    -37.85800    144.9005         6380.0  
Sunshine        -37.79880    144.8220         3755.0  
Glenroy         -37.70830    144.9158         8870.0  
Sunshine North  -37.76230    144.8272         4217.

#### Score from Approach 2 (Imputation)
Next, we use SimpleImputer to replace missing values with the mean value along each column.

Although it's simple, filling in the mean value generally performs quite well (but this varies by dataset). While statisticians have experimented with more complex ways to determine imputed values (such as regression imputation, for instance), the complex strategies typically give no additional benefit once you plug the results into sophisticated machine learning models.

In [6]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


In [7]:
print('imputed_X_train \n',imputed_X_train.head())
print('='*110)
print('imputed_X_valid \n',imputed_X_valid.head())
print('='*110)
print('imputed_X_train \n',imputed_X_train.isnull().sum())
print('='*110)
print('imputed_X_valid \n',imputed_X_valid.isnull().sum())

imputed_X_train 
    Rooms  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  \
0    1.0       5.0    3182.0       1.0       1.0  1.0       0.0    153.764119   
1    2.0       8.0    3016.0       2.0       2.0  1.0     193.0    153.764119   
2    3.0      12.6    3020.0       3.0       1.0  1.0     555.0    153.764119   
3    3.0      13.0    3046.0       3.0       1.0  1.0     265.0    153.764119   
4    3.0      13.3    3020.0       3.0       1.0  2.0     673.0    673.000000   

     YearBuilt  Lattitude  Longtitude  Propertycount  
0  1940.000000  -37.85984    144.9867        13240.0  
1  1964.839866  -37.85800    144.9005         6380.0  
2  1964.839866  -37.79880    144.8220         3755.0  
3  1995.000000  -37.70830    144.9158         8870.0  
4  1970.000000  -37.76230    144.8272         4217.0  
imputed_X_valid 
    Rooms  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  \
0    4.0       8.0    3016.0       4.0       2.0  2.0     450.0   

We see that Approach 2 has lower MAE than Approach 1, so Approach 2 performed better on this dataset.

### Score from Approach 3 (An Extension to Imputation)
Next, we impute the missing values, while also keeping track of which values were imputed.

In [8]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


As we can see, Approach 3 performed slightly worse than Approach 2.

#### So, why did imputation perform better than dropping the columns?
The training data has 10864 rows and 12 columns, where three columns contain missing data. For each column, less than half of the entries are missing. Thus, dropping the columns removes a lot of useful information, and so it makes sense that imputation would perform better.

In [9]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = X_train.isnull().sum()
print(missing_val_count_by_column[missing_val_count_by_column == 0])
print("="*110)
print(missing_val_count_by_column[missing_val_count_by_column >= 1])

(10864, 12)
Rooms            0
Distance         0
Postcode         0
Bedroom2         0
Bathroom         0
Landsize         0
Lattitude        0
Longtitude       0
Propertycount    0
dtype: int64
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


### Conclusion
As is common, imputing missing values (in Approach 2 and Approach 3) yielded better results, relative to when we simply dropped columns with missing values (in Approach 1).

## Exercise: Missing Values

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_full = pd.read_csv("Dataset/home-data-for-ml-course/train.csv", index_col = 0)
X_test_full = pd.read_csv("Dataset/home-data-for-ml-course/test.csv", index_col = 0)

# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8, test_size=0.2, random_state=0)

Use the next code cell to print the first five rows of the data.

In [11]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


You can already see a few missing values in the first several rows. In the next step, you'll obtain a more comprehensive understanding of the missing values in the dataset.

### Step 1: Preliminary investigation
Run the code cell below without changes.

In [12]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

(1168, 36)


#### Part A
Use the above output to answer the questions below.

In [13]:
# Fill in the line below: How many rows are in the training data?
num_rows = 1168

# Fill in the line below: How many columns in the training data
# have missing values?
num_cols_with_missing = 3

# Fill in the line below: How many missing entries are contained in 
# all of the training data?
tot_missing = 212 + 6 + 58

#### Part B
Considering your answers above, what do you think is likely the best approach to dealing with the missing values?

Solution: Since there are relatively few missing entries in the data (the column with the greatest percentage of missing values is missing less than 20% of its entries), we can expect that dropping columns is unlikely to yield good results. This is because we'd be throwing away a lot of valuable data, and so imputation will likely perform better.

To compare different approaches to dealing with missing values, you'll use the same score_dataset() function from the tutorial. This function reports the mean absolute error (MAE) from a random forest model.

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators = 100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### Step 2: Drop columns with missing values
In this step, you'll preprocess the data in X_train and X_valid to remove columns with missing values. Set the preprocessed DataFrames to reduced_X_train and reduced_X_valid, respectively.

In [15]:
# Fill in the line below: get names of columns with missing values
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Fill in the lines below: drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

Run the next code cell without changes to obtain the MAE for this approach.

In [16]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


### Step 3: Imputation
#### Part A
Use the next code cell to impute missing values with the mean value along each column. Set the preprocessed DataFrames to imputed_X_train and imputed_X_valid. Make sure that the column names match those in X_train and X_valid.

In [17]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

Run the next code cell without changes to obtain the MAE for this approach.

In [18]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18062.894611872147


#### Part B
Compare the MAE from each approach. Does anything surprise you about the results? Why do you think one approach performed better than the other?

Solution: Given that thre are so few missing values in the dataset, we'd expect imputation to perform better than dropping columns entirely. However, we see that dropping columns performs slightly better! While this can probably partially be attributed to noise in the dataset, another potential explanation is that the imputation method is not a great match to this dataset. That is, maybe instead of filling in the mean value, it makes more sense to set every missing value to a value of 0, to fill in the most frequently encountered value, or to use some other method. For instance, consider the GarageYrBlt column (which indicates the year that the garage was built). It's likely that in some cases, a missing value could indicate a house that does not have a garage. Does it make more sense to fill in the median value along each column in this case? Or could we get better results by filling in the minimum value along each column? It's not quite clear what's best in this case, but perhaps we can rule out some options immediately - for instance, setting missing values in this column to 0 is likely to yield horrible results!

### Step 4: Generate test predictions
In this final step, you'll use any approach of your choosing to deal with missing values. Once you've preprocessed the training and validation features, you'll train and evaluate a random forest model. Then, you'll preprocess the test data before generating predictions that can be submitted to the competition!

#### Part A
Use the next code cell to preprocess the training and validation data.  Set the preprocessed DataFrames to `final_X_train` and `final_X_valid`.  **You can use any approach of your choosing here!**  in order for this step to be marked as correct, you need only ensure:
- the preprocessed DataFrames have the same number of columns,
- the preprocessed DataFrames have no missing values, 
- `final_X_train` and `y_train` have the same number of rows, and
- `final_X_valid` and `y_valid` have the same number of rows.

In [19]:
# Preprocessed training and validation features
final_X_train = imputed_X_train
final_X_valid = imputed_X_valid

Run the next code cell to train and evaluate a random forest model. (Note that we don't use the score_dataset() function above, because we will soon use the trained model to generate test predictions!)

In [20]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))
# print(score_dataset(final_X_train, final_X_valid, y_train, y_valid))

MAE (Your approach):
18062.894611872147


#### Part B

Use the next code cell to preprocess your test data.  Make sure that you use a method that agrees with how you preprocessed the training and validation data, and set the preprocessed test features to `final_X_test`.

Then, use the preprocessed test features and the trained model to generate test predictions in `preds_test`.

In order for this step to be marked correct, you need only ensure:
- the preprocessed test DataFrame has no missing values, and
- `final_X_test` has the same number of rows as `X_test`.

In [21]:
# Fill in the line below: preprocess test data
final_imputer = SimpleImputer()
final_X_test = pd.DataFrame(final_imputer.fit_transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)

In [22]:
# Save test predictions to file
# output = pd.DataFrame({'Id': X_test.index,
#                        'SalePrice': preds_test})
# output.to_csv('submission.csv', index=False)